In [1]:
%pip install toolz
%pip install gradio

import json
import pandas as pd


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
def read_raw_git_file(url):
    """Reads a raw file from github and returns the content as a string."""
    import requests
    response = requests.get(url)
    return response.text

In [ ]:
#'https://raw.githubusercontent.com/QuantLet/Encode-the-Qode/main/BART_learning/BART.ipynb?token=GHSAT0AAAAAABQSSIVUV74YDZCRY3ZI6BC2ZB4IJCA'
response = read_raw_git_file('https://raw.githubusercontent.com/QuantLet/Encode-the-Qode/main/BART_learning/BART.ipynb?token=GHSAT0AAAAAABQSSIVUV74YDZCRY3ZI6BC2ZB4IJCA')


In [ ]:
import gradio as gr
from transformers import pipeline, SummarizationPipeline, AutoModelWithLMHead, AutoTokenizer

generator = SummarizationPipeline(
    model=AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_large_source_code_summarization_python_multitask"),
    tokenizer=AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_large_source_code_summarization_python_multitask",
    skip_special_tokens=False),
    device=0
)

def read_raw_git_file(url):

    """Reads a raw file from github and returns the content as a string."""
    
    import requests
    response = requests.get(url)
    return response.text

def generate(url):
    import json
    CLEAN_UP = True
    response = read_raw_git_file(url)
    data = json.loads(response)
    data = pd.json_normalize(data, record_path='cells')

    data = data[data['cell_type'].isin(['code', 'markdown'])]
    data = data[data['source'].notna()]
    data = data[data['source'].apply(lambda x: len(x) > 0)]

    data = [item for sublist in data.source.values for item in sublist]
    data = [item.replace('\n', '') for item in data if item.endswith('\n')]
    data = [item for item in data if len(item)>0]

    samples = [f'{item}\n' for item in data]

    if isinstance(samples, str):
        nl_description = generator([samples])
    else:
        nl_description = generator([codeline for codeline in samples])
    #result = generator(text, max_length=30, num_return_sequences=1)
    #nl_description = pipeline([codeline[0] for codeline in samples])
    if CLEAN_UP:
            nl_description = [re.sub(r'[^a-zA-Z0-9\s]', '', summary['summary_text']) for summary in nl_description]
            nl_description = [re.sub(r'\s+', ' ', summary).strip() for summary in nl_description]
            nl_description = [summary for summary in nl_description if len(summary) > 0]
    return nl_description


demo = gr.Interface(
    fn=generate,
    inputs=gr.inputs.Textbox(lines=5, label="Input Code URL"),
    outputs=gr.outputs.Textbox(label="Generated Description"),
)

demo.launch(share=True)

/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/home/RDC/zino

Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://7c3df94e8321134a8b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
